In [ ]:
import cadquery as cq
from jupyter_cadquery import (
    open_viewer, show,
    set_defaults, get_defaults
)
from cadquery_massembly import Mate, MAssembly, relocate

cv = open_viewer("Hexapod", cad_width=800, height=600, theme="light")

In [ ]:
set_defaults(axes=False, axes0=True, mate_scale=5, transparent=False)

# Hexapod 

![2-hexapod.png](2-hexapod.png)

In [ ]:
import numpy as np
horizontal_angle = 25

def intervals(count):
    r = [ min(180, (90 + i*(360 // count)) % 360) for i in range(count)]
    return r 

def times(end, count):
    return np.linspace(0, end, count+1)
    
def vertical(count, end, offset, reverse):
    ints = intervals(count)
    heights = [round(35 * np.sin(np.deg2rad(x)) - 15, 1) for x in ints]
    heights.append(heights[0])
    return times(end, count), heights[offset:] + heights[1:offset+1]

def horizontal(end, reverse):
    factor = 1 if reverse else -1
    return times(end, 4), [0, factor * horizontal_angle, 0, -factor * horizontal_angle, 0]

print("Leg group 1 (transparent)")
print("horizontal movement    ", horizontal(4, True))
print("vertical heights (left) ", vertical(8, 4, 0, True))
print("vertical heights (right)", vertical(8, 4, 0, False))

print("\nLeg group 1 (filled)")
print("horizontal movement", horizontal(4, False))
print("vertical heights (left) ", vertical(8, 4, 4, True))
print("vertical heights (right)", vertical(8, 4, 4, False))


# Assembly

## Parts

In [ ]:
thickness = 2
height = 40
width = 65
length = 100
diam = 4
tol = 0.05

In [ ]:
def create_base():
    x1, x2 = 0.63, 0.87
    base_holes = {
        "right_front": (-x1*width, -x1*length), "right_middle": (-x2*width, 0), "right_back": ( -x1*width, x1*length ),
        "left_front":  ( x1*width, -x1*length), "left_middle":  ( x2*width, 0), "left_back":  (  x1*width, x1*length ),
    }
    stand_holes = {"front_stand": (0, -0.75 * length), "back_stand": (0, 0.8 * length)}

    workplane = cq.Workplane()

    base = (workplane
        .ellipseArc(width, length, 295, 245, startAtCurrent=False).close()
        .pushPoints(list(base_holes.values())).circle(diam / 2 + tol)
        .moveTo(*stand_holes["back_stand" ]).rect(width / 2 + 2 * tol, thickness + 2 * tol)
        .moveTo(*stand_holes["front_stand"]).rect(width / 2 + 2 * tol, thickness + 2 * tol)
        .extrude(thickness)
    )
    base


    base.faces("<Z").tag("bottom")
    base.faces(">Z").tag("top")

    for name, hole in base_holes.items():
        base.faces("<Z").wires(cq.NearestToPointSelector(hole)).tag(name)

    for name, hole in stand_holes.items():
        base.faces("<Z").wires(cq.NearestToPointSelector(hole)).tag(name)

    return base

base_holes_names = {"right_back", "right_middle", "right_front", "left_back", "left_middle", "left_front"}

In [ ]:
def create_stand():
    stand = cq.Workplane().box(height, width / 2 + 10, thickness)
    inset = cq.Workplane().box(thickness, width / 2, thickness)
    backing = cq.Workplane("ZX").polyline([(10,0), (0,0), (0, 10)]).close().extrude(thickness)

    stand = (stand
        .union(inset.translate(( (height + thickness) / 2, 0, 0)))
        .union(inset.translate((-(height + thickness) / 2, 0, 0)))
        .union(backing.translate((-height / 2, -thickness / 2, thickness / 2)))
        .union(backing.rotate((0, 0, 0), (0, 1, 0), -90).translate((height / 2, -thickness / 2, thickness / 2)))
    )
    return stand

stand_names = ("front_stand", "back_stand")

In [ ]:
def create_upper_leg():
    l1, l2 = 50, 80
    pts = [( 0,  0), ( 0, height/2), (l1, height/2 - 5), (l2, 0)]
    upper_leg_hole = (l2 - 10, 0)

    upper_leg = (cq.Workplane()
        .polyline(pts).mirrorX()
        .pushPoints([upper_leg_hole]).circle(diam/2 + tol).extrude(thickness)
        .edges("|Z and (not <X)").fillet(4)
    )

    axle = (cq.Workplane("XZ", origin=(0, height/2 + thickness + tol, thickness/2))
        .circle(diam/2).extrude(2 * (height/2 + thickness + tol))
    )

    upper_leg = upper_leg.union(axle)

    # tag mating points
    upper_leg.faces(">Z").edges(cq.NearestToPointSelector(upper_leg_hole)).tag("top")
    upper_leg.faces("<Z").edges(cq.NearestToPointSelector(upper_leg_hole)).tag("bottom")

    return upper_leg

def create_lower_leg():
    w, l1, l2 = 15, 20, 120
    pts = [( 0,  0), ( l1, w), (l2, 0)]
    lower_leg_hole = (l1 - 10, 0)

    lower_leg = (cq.Workplane()
        .polyline(pts).mirrorX()
        .pushPoints([lower_leg_hole]).circle(diam/2 + tol)
        .extrude(thickness)
        .edges("|Z").fillet(5)
    )

    # tag mating points
    lower_leg.faces(">Z").edges(cq.NearestToPointSelector(lower_leg_hole)).tag("top"),
    lower_leg.faces("<Z").edges(cq.NearestToPointSelector(lower_leg_hole)).tag("bottom")
    
    return lower_leg

leg_angles = {
    "right_back": -105,  "right_middle":-90, "right_front":-75, 
    "left_back":   105,  "left_middle":  90, "left_front":  75,
    
}
leg_names = list(leg_angles.keys())

In [ ]:
base = create_base()
stand = create_stand()
upper_leg = create_upper_leg()
lower_leg = create_lower_leg()

cv = show(
    base, 
    stand.translate((100, -60, thickness/2)), 
    upper_leg.translate((80,0,0)),
    lower_leg.translate((75,60,0)),
)

## Define Assembly

In [ ]:
def create_hexapod():
    # Some shortcuts
    L = lambda *args: cq.Location(cq.Vector(*args))

    # Leg assembly
    leg = (MAssembly(upper_leg, name="upper", color="orange")
        .add(lower_leg, name="lower", color="orange", loc=L(80,0,0))
    )
    # Hexapod assembly
    hexapod = (MAssembly(base, name="bottom", color="silver")
        .add(base, name="top", color="gainsboro", loc=L(0, -2*length, 0))
        .add(stand, name="front_stand", color="SkyBlue", loc=L(120, -255, 0))
        .add(stand, name="back_stand", color="SkyBlue", loc=L(180, -255, 0))
    )
    
    for i, name in enumerate(leg_names):
        hexapod.add(leg, name=name, loc=L(100, -55*(i-1.4), 0))

    return hexapod

## Define Mates 

In [ ]:
from collections import OrderedDict as odict

hexapod = create_hexapod()

hexapod.mate("bottom?top", name="bottom", origin=True)
hexapod.mate("top?bottom", name="top", origin=True, transforms=odict(rx=180, tz=-(height + 2 * tol)))

for name in stand_names:
    hexapod.mate(f"bottom?{name}", name=f"{name}_bottom", transforms=odict(rz=180 if "f" in name else 0))
    hexapod.mate(f"{name}@faces@<X", name=name, origin=True, transforms=odict(rx=180))

for name in base_holes_names:
    hexapod.mate(f"bottom?{name}", name=f"{name}_hole", transforms=odict(rz=leg_angles[name]))

for name in leg_names:
    lower, upper, angle = ("top", "bottom", -75) if "left" in name else ("bottom", "top", -75)
    hexapod.mate(f"{name}?{upper}", name=f"leg_{name}_hole", transforms=odict(rz=angle))
    hexapod.mate(f"{name}@faces@<Y", name=f"leg_{name}_hinge", origin=True, transforms=odict(rx=180))
    hexapod.mate(f"{name}/lower?{lower}", name=f"leg_{name}_lower_hole", origin=True)

cv = show(hexapod, render_mates=True)

## Relocate and assemble

In [ ]:
hexapod.relocate()
cv = show(hexapod)

In [ ]:
for leg in leg_names:
    hexapod.assemble(f"leg_{leg}_lower_hole", f"leg_{leg}_hole")
    hexapod.assemble(f"leg_{leg}_hinge", f"{leg}_hole")

hexapod.assemble("top", "bottom")

for stand_name in stand_names:
    hexapod.assemble(f"{stand_name}", f"{stand_name}_bottom")


cv = show(hexapod, render_mates=False, grid=[True, False, False], axes=False, render_normals=False)

## Animation

In [ ]:
from jupyter_cadquery import AnimationTrack

leg_group = ("left_front", "right_middle", "left_back")

for name in leg_names:
    # move upper leg
    cv.add_track(AnimationTrack(
        f"/bottom/{name}", 
        "rz", *horizontal(4, "middle" in name)
    ))

    # move lower leg
    cv.add_track(AnimationTrack(
        f"/bottom/{name}/lower", 
        "rz", *vertical(8, 4, 0 if name in leg_group else 4, "left" in name)
    ))
    
    # lift hexapod to run on grid
    # animation.add_track(f"/bottom", "tz", [0, 4], [-118]*2)
    
cv.animate(speed=2)

In [ ]:
# close_viewers()

In [ ]:
from build123d import *

In [ ]:
t = Text("sidecar", 20)
cv_s = show(t, viewer="SideCar", anchor="right")

In [ ]:
t = Text("right", 20)
cv_r = show(t, viewer="Right", anchor="split-right")

In [ ]:
t = Text("left", 20)
cv_l = show(t, viewer="Left", anchor="split-left")

In [ ]:
cv_t = open_viewer("Top", anchor="split-top", aspect_ratio=0)
t = Text("top", 20)
show(t)

In [ ]:
cv_b = open_viewer("Bottom", anchor="split-bottom", aspect_ratio=0)
t = Text("bottom", 20)
show(t)